In [24]:
import param
import panel as pn
import subprocess
import pandas as pd
from Bio import SeqIO, AlignIO
from io import StringIO
#import ray
import os
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqUtils import gc_fraction
from glob import glob
import csv
from collections import Counter
#import plotly.graph_objs as go
#import plotly.express as px
import numpy as np
from bokeh.models import ColumnDataSource, Plot, Grid, Range1d
from bokeh.plotting import figure
from bokeh.models.glyphs import Text, Rect
from bokeh.layouts import gridplot
import pyperclip as pc

In [25]:
project_folder = "/home/infosebi/Documents/Programmieren/MastersProject/test_3"
mafft_folder=f"{project_folder}/aligned"
tree_folder=f"{project_folder}/trees"
blast_folder = f"{project_folder}/diamond_blast"

In [26]:
pn.extension('tabulator')
pipeline = pn.pipeline.Pipeline(debug=True)

In [30]:
class Blast(param.Parameterized):
    def view(self):
        
        ###########
        os.chdir(project_folder)
        #############

        def get_colors(seqs, protein):
            """make colors for bases in sequence"""
            text = [i for s in list(seqs) for i in s]
            if protein == True:
                clrs =	{'A': 'lightgreen', 'G': 'lightgreen', 'C': 'green', 'D': 'darkgreen', 'E': 'darkgreen', 'N': 'darkgreen', 'Q': 'darkgreen',
                        'I': 'blue', 'L': 'blue', 'M': 'blue', 'V': 'blue', 'F': 'palevioletred', 'W': ' palevioletred', 'Y': ' palevioletred', 'H': 'darkblue', 'K': 'orange',
                        'R': 'orange', 'P': 'pink', 'S': 'red', 'T': 'red', '-': 'white'}
            else:
                clrs =  {'A':'red','T':'green','G':'orange','C':'blue','-':'white'}
            colors = [clrs[i.upper()] for i in text]
            return colors
        
        
        def view_alignment(aln, protein_bool, fontsize="9pt", plot_width=800):
            """Bokeh sequence alignment view"""
        
            #make sequence and id lists from the aln object
            seqs = [rec.seq for rec in (aln)]
            ids = [rec.id for rec in aln]    
            text = [i for s in list(seqs) for i in s.upper()]
            colors = get_colors(seqs, protein_bool)    
            N = len(seqs[0])
            S = len(seqs)    
            width = .4
        
            x = np.arange(1,N+1)
            y = np.arange(0,S,1)
            #creates a 2D grid of coords from the 1D arrays
            xx, yy = np.meshgrid(x, y)
            #flattens the arrays
            gx = xx.ravel()
            gy = yy.flatten()
            #use recty for rect coords with an offset
            recty = gy+.5
            h= 1/S
            #now we can create the ColumnDataSource with all the arrays
            source = ColumnDataSource(dict(x=gx, y=gy, recty=recty, text=text, colors=colors ))#colors=colors
            plot_height = len(seqs)*15+50
            x_range = Range1d(0,N+1, bounds='auto')
            if N>100:
                viewlen=100
            else:
                viewlen=N
            #view_range is for the close up view
            view_range = (0,viewlen)
            tools="xpan, xwheel_zoom, reset, save"
        
            #entire sequence view (no text, with zoom)
            p = figure(title=None,  height=50, width=plot_width,   #plot_width= plot_width,
                       x_range=x_range, y_range=(0,S), tools=tools,
                       min_border=0, toolbar_location='below')
            rects = Rect(x="x", y="recty",  width=1, height=1, fill_color="colors",
                         line_color=None, fill_alpha=0.6)
            p.add_glyph(source, rects)
            p.yaxis.visible = False
            p.grid.visible = False  
        
            #sequence text view with ability to scroll along x axis
            p1 = figure(title=None, width=plot_width, height=plot_height,
                        x_range=view_range, y_range=ids, tools="xpan,reset",
                        min_border=0, toolbar_location='below')#, lod_factor=1)          
            glyph = Text(x="x", y="y", text="text", text_align='center',text_color="black", text_font_size=fontsize)
            rects = Rect(x="x", y="recty",  width=1, height=1, fill_color="colors",
                        line_color=None, fill_alpha=0.4)
            p1.add_glyph(source, glyph)
            p1.add_glyph(source, rects)
        
            p1.grid.visible = False
            p1.xaxis.major_label_text_font_style = "bold"
            p1.yaxis.minor_tick_line_width = 0
            p1.yaxis.major_tick_line_width = 0
        
            p = gridplot([[p],[p1]], toolbar_location='below')
            return p
        
        def blast(event):
            ###########
            os.chdir(project_folder)
            ###########
            blastmode = 'blastp'
            if blast_switch.value == True:
                blastmode = 'blastx'
            with open("querie.fasta", "w") as output_handle:
                SeqIO.write(SeqRecord(Seq(seq_input.value), id=id_input.value, description=''), output_handle, "fasta")
            print(f"diamond {blastmode} -d clusterdb -q querie.fasta -o diamond_blast/{id_input.value}.tsv")
            subprocess.run(f"diamond {blastmode} -d clusterdb -q querie.fasta -o diamond_blast/{id_input.value}.tsv",shell=True)
            os.remove('querie.fasta')
            results_df = pd.read_csv(f"{blast_folder}/{id_input.value}.tsv", sep='\t', index_col=1, names=['Query accession', 'Target accession', 'Sequence identity', 'Length', 'Mismatches', 'Gap openings', 'Query start', 'Query end', 'Target start', 'Target end', "E-value", "Bit score"])
            results_tabulator.value = results_df
            protein_df = pd.read_csv('genome_to_protein.csv', index_col=0)
            cluster_df = pd.read_csv('cluster_to_protein.csv', index_col=0)
            recluster_df = pd.read_csv('reclusters.txt', sep='\t', index_col=0, names=['Cluster', 'Protein'])
            matching_sequence_list = []
            if not results_df.empty:
                align = []
                align.append(SeqRecord(Seq(seq_input.value), id=id_input.value, description=''))
                for index, row in results_df.iterrows():
                    cluster = recluster_df.index[recluster_df['Protein'] == index].tolist()[0]
                    sequence = protein_df.loc[index]['sequence']
                    matching_sequence_row = {'Name': index, 'Cluster': cluster, 'Pangenome': cluster_df.loc[cluster]['pangenome'],'Sequence': sequence, 'Product': protein_df.loc[index]['product'], 'Type': protein_df.loc[index]['type'], }
                    matching_sequence_list.append(matching_sequence_row)
                    align.append(SeqRecord(Seq(protein_df.loc[index]['sequence']), id=index, description=''))
                matching_sequence_df = pd.DataFrame.from_dict(matching_sequence_list)
                sequence_tabulator.value = matching_sequence_df
                out_handle = StringIO()
                SeqIO.write(align, out_handle, "fasta")
                fasta_data = out_handle.getvalue()
                mafft = subprocess.Popen(["mafft", "--auto", "--thread", "-1", "-"], stdout=subprocess.PIPE, stdin=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
                stdout_data, stderr_data = mafft.communicate(input=fasta_data)
                output_file = f'{blast_folder}/{id_input.value}_alignment.fasta'
                with open(output_file, 'w') as output_handle:
                        output_handle.write(stdout_data)
                protein_bool = True
                if blast_switch.value == True:
                    protein_bool = False
                aln = AlignIO.read(output_file,'fasta')
                alignment_viewer = view_alignment(aln=aln, protein_bool=protein_bool ,plot_width=900)
                alignment_viewer_bokeh.object = alignment_viewer

        
        id_input = pn.widgets.TextInput(name='Name of query', placeholder='Enter an identifier for you blast search...')
        seq_input = pn.widgets.TextInput(name='Sequence', placeholder='Enter Sequence...')
        blast_switch = pn.widgets.Switch(name='Switch')
        protein_text = pn.widgets.StaticText(value='Amino acids')
        nucleic_text = pn.widgets.StaticText(value='Nucleic acids')
        
        blast_button = pn.widgets.Button(name='Diamond BLAST', button_type='primary')
        blast_button.on_click(blast)
        
        results_tabulator = pn.widgets.Tabulator()
        sequence_tabulator = pn.widgets.Tabulator()
        alignment_viewer_bokeh = pn.pane.Bokeh()
        
        blast_interface = pn.Column(pn.Row(id_input, seq_input, pn.Row(protein_text, blast_switch, nucleic_text), blast_button), results_tabulator, sequence_tabulator, alignment_viewer_bokeh)
        return blast_interface
    def panel(self):
        return self.view()

In [31]:
pipeline.add_stage('Diamond Blast', Blast)

In [32]:
pipeline.servable()

>my_prot 
MKEFNVYLNSIDKVKNFVEKMSKYPFDIDIVSGRYVIDAKSIMGIFSLNLENQLTVIPHT 
EDEEALSKFSDDIKEYIA 
>J2Z80_001709 
MKEFNVYLNSIDKVKNFVEKMSKYPFDIDIVSGRYVIDAKSIMGIFSLNLENQLTVIPHT 
EDEEALSKFSDDIKEYIA 
>V518_1966 
MKEFQVSLNSIDKVKTFVEKMSKYPFDIDIVSGRYVIDAKSIMGIFSLNLENLLTVIPHT 
DDEETLAKFSDDIKEYMK 
>prot_5255.1_2511 
MKEFQVSLNSIDKVKTFVEKMSKYPFDIDIVSGRYVIDAKSIMGIFSLNLENLLTVIPHT 
DDEETLAKFSDDIKEYMK* 
 
mafft?

Column(sizing_mode='stretch_width')
    [0] Row(sizing_mode='stretch_width')
        [0] Column
            [0] Markdown(str, margin=(0, 0, 0, 5))
            [1] Row(width=100)
        [1] HoloViews(Overlay, backend='bokeh', height=60, sizing_mode='stretch_width')
        [2] Row
            [0] Button(disabled=True, name='Previous', width=125)
            [1] Button(disabled=True, name='Next', width=125)
    [1] Row
        [0] Column
            [0] Row
                [0] TextInput(name='Name of query', placeholder='Enter an identifier f...)
                [1] TextInput(name='Sequence', placeholder='Enter Sequence...')
                [2] Row
                    [0] StaticText(value='Amino acids')
                    [1] Switch()
                    [2] StaticText(value='Nucleic acids')
                [3] Button(button_type='primary', name='Diamond BLAST')
            [1] Tabulator()
            [2] Tabulator()
            [3] Bokeh(None)

In [ ]:
#prot_5255.1_1466